In [1]:
import pandas as pd
import re
import warnings
from fuzzywuzzy import fuzz
warnings.filterwarnings('ignore')
pd.set_option('max_columns',151)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
outfield = pd.read_csv('data/2019-20_out.csv')
fpl = pd.read_csv('data/cleaned_players.csv')

In [3]:
for c in outfield.columns:
    print(f"'{c}'",end=',')

'player','nationality','position','squad','age','birth_year','games','games_starts','minutes','goals','assists','pens_made','pens_att','cards_yellow','cards_red','goals_per90','assists_per90','goals_assists_per90','goals_pens_per90','goals_assists_pens_per90','xg','npxg','xa','xg_per90','xa_per90','xg_xa_per90','npxg_per90','npxg_xa_per90','minutes_90s','shots_total','shots_on_target','shots_free_kicks','shots_on_target_pct','shots_total_per90','shots_on_target_per90','goals_per_shot','goals_per_shot_on_target','npxg_per_shot','xg_net','npxg_net','passes_completed','passes','passes_pct','passes_total_distance','passes_progressive_distance','passes_completed_short','passes_short','passes_pct_short','passes_completed_medium','passes_medium','passes_pct_medium','passes_completed_long','passes_long','passes_pct_long','xa_net','assisted_shots','passes_into_final_third','passes_into_penalty_area','crosses_into_penalty_area','progressive_passes','passes_live','passes_dead','passes_free_kicks'

In [4]:
defender = outfield.loc[:,['player','position','squad','games','minutes','cards_yellow','cards_red','goals_per90',
                           'assists_per90','goals_assists_per90','xg_per90','xa_per90','xg_xa_per90',
                           'tackles','tackles_won','dribble_tackles_pct','pressure_regain_pct',
                           'blocks','interceptions','clearances']]
defender = defender[defender['position']=='DF']
#relegated = ['Bournemouth','Norwich City','Watford']
#defender = defender[~defender['squad'].isin(relegated)]
defender['tackle_perc'] = None
defender['block_int_clear'] = None
defender['tackle_perc'] = defender.apply(lambda row: (row['tackles_won']/row['tackles']) if row['tackles']!=0 else 0, axis=1)
defender['block_int_clear'] = defender.apply(lambda row:(row['blocks']+row['interceptions']+row['clearances']),axis=1)
defender = defender.drop(columns=['tackles','tackles_won','blocks','interceptions','clearances']).reset_index(drop=True)
defender.head()

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0
1,Max Aarons,DF,Norwich City,36.0,3240.0,7.0,0.0,0.00,0.03,0.03,0.01,0.09,0.11,46.7,27.2,0.562500,221.0
2,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0
3,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0
4,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0


In [5]:
dup_def = defender[defender.duplicated(['player'],keep=False)]
dup_def = dup_def.groupby(['player'], as_index=False).agg('sum')
dup_def.insert(1,'position','DF')
dup_squad = ['Southampton + Arsenal','Newcastle Utd + Tottenham','Tottenham + Southampton','Wolves + Leicester City','Chelsea + Brighton']
dup_def.insert(2,'squad',dup_squad)
dup_def

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Cédric Soares,DF,Southampton + Arsenal,21.0,1555.0,1.0,0.0,0.36,0.07,0.43,0.05,0.11,0.15,87.4,58.2,1.355072,129.0
1,Danny Rose,DF,Newcastle Utd + Tottenham,23.0,1756.0,7.0,0.0,0.00,0.00,0.00,0.02,0.09,0.10,86.2,64.5,1.389423,105.0
2,Kyle Walker-Peters,DF,Tottenham + Southampton,13.0,944.0,1.0,0.0,0.00,0.00,0.00,0.02,0.10,0.12,53.3,62.9,1.171429,82.0
3,Ryan Bennett,DF,Wolves + Leicester City,16.0,937.0,2.0,0.0,0.00,0.00,0.00,0.00,0.02,0.02,95.3,44.6,1.150000,94.0
4,Tariq Lamptey,DF,Chelsea + Brighton,9.0,620.0,2.0,0.0,0.00,0.15,0.15,0.01,0.43,0.44,100.0,118.7,0.333333,46.0


In [6]:
defender = defender.drop_duplicates(subset='player',keep=False)
defender = pd.concat([defender,dup_def])
defender.head()

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0
1,Max Aarons,DF,Norwich City,36.0,3240.0,7.0,0.0,0.00,0.03,0.03,0.01,0.09,0.11,46.7,27.2,0.562500,221.0
2,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0
3,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0
4,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0


In [7]:
defender.shape

(176, 17)

In [8]:
fpl

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type
0,Mesut,Özil,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,1.0,69,MID
1,Sokratis,Papastathopoulos,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.1,49,DEF
2,David,Luiz Moreira Marinho,0,0,1,1,0,0.0,0.0,0.0,0,3,0.0,0,0,0,0.9,55,DEF
3,Pierre-Emerick,Aubameyang,1,1,12,180,1,51.7,64.6,69.0,0,39,18.5,1,0,1,39.5,120,MID
4,Cédric,Soares,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.4,49,DEF
5,Alexandre,Lacazette,2,0,14,162,1,18.4,71.6,79.0,2,58,16.9,1,0,0,5.3,85,FWD
6,Shkodran,Mustafi,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,50,DEF
7,Bernd,Leno,0,0,9,180,1,0.0,32.0,0.0,1,47,3.2,1,0,0,9.4,50,GK
8,Granit,Xhaka,0,0,5,167,1,23.9,23.2,21.0,0,21,6.8,1,0,0,0.9,55,MID
9,Pablo,Marí,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0.0,45,DEF


In [9]:
fpl_def = fpl[fpl['element_type']=='DEF']
fpl_def['cost'] = None
fpl_def['player'] = None
fpl_def['player'] = fpl_def['first_name'].str.cat(fpl_def['second_name'],sep=' ')

fpl_def = fpl_def.loc[:,['second_name','player','total_points','now_cost']].reset_index(drop=True)
fpl_def.head()

,second_name,player,total_points,now_cost
0,Papastathopoulos,Sokratis Papastathopoulos,0,49
1,Luiz Moreira Marinho,David Luiz Moreira Marinho,1,55
2,Soares,Cédric Soares,0,49
3,Mustafi,Shkodran Mustafi,0,50
4,Marí,Pablo Marí,0,45


In [10]:
cost = {}
points = {}

for def_p in defender.itertuples():
    for fpl_p in fpl_def.itertuples():
        if (fuzz.token_set_ratio(def_p[1],fpl_p[2])>80):
            points[def_p[1]] = fpl_p[3]
            cost[def_p[1]] = fpl_p[4]/10
            break

Patrick van Aanholt 0 5.5
Nathan Aké 2 5.5
Toby Alderweireld 2 5.5
Trent Alexander-Arnold 8 7.5
Marcos Alonso 3 6.0
Serge Aurier 0 5.4
César Azpilicueta 1 5.9
Eric Bailly 0 5.0
Fabián Balbuena 0 4.5
George Baldock 3 5.5
Phil Bardsley -1 4.5
Chris Basham 3 5.0
Jan Bednarek 1 4.5
Héctor Bellerín 7 5.0
Bernardo 0 4.0
Ryan Bertrand 1 5.0
Willy Boly 7 5.5
Jarrad Branthwaite 0 4.0
Dan Burn 1 4.5
Gary Cahill 0 4.5
João Cancelo 0 5.5
Calum Chambers 0 4.5
Ben Chilwell 0 5.5
Andreas Christensen -1 5.0
Ciaran Clark 0 4.5
Conor Coady 9 5.0
Séamus Coleman 7 5.0
Aaron Cresswell 2 5.0
Diogo Dalot 0 5.0
Scott Dann 7 4.5
Ben Davies 3 5.0
Lucas Digne 13 6.1
Virgil van Dijk 14 6.5
Issa Diop 2 4.5
Matt Doherty 2 6.0
Shane Duffy 0 4.5
Paul Dummett 0 4.5
Lewis Dunk 8 5.0
John Egan -2 5.0
Ahmed Elmohamady 0 4.5
Björn Engels 0 4.5
Jonny Evans 0 5.5
Federico Fernández 7 5.0
Timothy Fosu-Mensah 0 4.5
Juan Foyth 0 4.5
Ryan Fredericks 3 4.5
Christian Fuchs 0 4.5
Eric García 0 5.0
Joe Gomez 1 5.5
Frederic Guilbert

In [11]:
points

{'Patrick van Aanholt': 0,
 'Nathan Aké': 2,
 'Toby Alderweireld': 2,
 'Trent Alexander-Arnold': 8,
 'Marcos Alonso': 3,
 'Serge Aurier': 0,
 'César Azpilicueta': 1,
 'Eric Bailly': 0,
 'Fabián Balbuena': 0,
 'George Baldock': 3,
 'Phil Bardsley': -1,
 'Chris Basham': 3,
 'Jan Bednarek': 1,
 'Héctor Bellerín': 7,
 'Bernardo': 0,
 'Ryan Bertrand': 1,
 'Willy Boly': 7,
 'Jarrad Branthwaite': 0,
 'Dan Burn': 1,
 'Gary Cahill': 0,
 'João Cancelo': 0,
 'Calum Chambers': 0,
 'Ben Chilwell': 0,
 'Andreas Christensen': -1,
 'Ciaran Clark': 0,
 'Conor Coady': 9,
 'Séamus Coleman': 7,
 'Aaron Cresswell': 2,
 'Diogo Dalot': 0,
 'Scott Dann': 7,
 'Ben Davies': 3,
 'Lucas Digne': 13,
 'Virgil van Dijk': 14,
 'Issa Diop': 2,
 'Matt Doherty': 2,
 'Shane Duffy': 0,
 'Paul Dummett': 0,
 'Lewis Dunk': 8,
 'John Egan': -2,
 'Ahmed Elmohamady': 0,
 'Björn Engels': 0,
 'Jonny Evans': 0,
 'Federico Fernández': 7,
 'Timothy Fosu-Mensah': 0,
 'Juan Foyth': 0,
 'Ryan Fredericks': 3,
 'Christian Fuchs': 0,
 'Er

In [12]:
cost

{'Patrick van Aanholt': 5.5,
 'Nathan Aké': 5.5,
 'Toby Alderweireld': 5.5,
 'Trent Alexander-Arnold': 7.5,
 'Marcos Alonso': 6.0,
 'Serge Aurier': 5.4,
 'César Azpilicueta': 5.9,
 'Eric Bailly': 5.0,
 'Fabián Balbuena': 4.5,
 'George Baldock': 5.5,
 'Phil Bardsley': 4.5,
 'Chris Basham': 5.0,
 'Jan Bednarek': 4.5,
 'Héctor Bellerín': 5.0,
 'Bernardo': 4.0,
 'Ryan Bertrand': 5.0,
 'Willy Boly': 5.5,
 'Jarrad Branthwaite': 4.0,
 'Dan Burn': 4.5,
 'Gary Cahill': 4.5,
 'João Cancelo': 5.5,
 'Calum Chambers': 4.5,
 'Ben Chilwell': 5.5,
 'Andreas Christensen': 5.0,
 'Ciaran Clark': 4.5,
 'Conor Coady': 5.0,
 'Séamus Coleman': 5.0,
 'Aaron Cresswell': 5.0,
 'Diogo Dalot': 5.0,
 'Scott Dann': 4.5,
 'Ben Davies': 5.0,
 'Lucas Digne': 6.1,
 'Virgil van Dijk': 6.5,
 'Issa Diop': 4.5,
 'Matt Doherty': 6.0,
 'Shane Duffy': 4.5,
 'Paul Dummett': 4.5,
 'Lewis Dunk': 5.0,
 'John Egan': 5.0,
 'Ahmed Elmohamady': 4.5,
 'Björn Engels': 4.5,
 'Jonny Evans': 5.5,
 'Federico Fernández': 5.0,
 'Timothy Fosu

In [13]:
defender['cost'] = defender['player'].map(cost)
defender['points'] = defender['player'].map(points)
defender = defender.reset_index(drop=True)
defender

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0,5.5,0.0
1,Max Aarons,DF,Norwich City,36.0,3240.0,7.0,0.0,0.00,0.03,0.03,0.01,0.09,0.11,46.7,27.2,0.562500,221.0,NaN,NaN
2,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0,5.5,2.0
3,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0,5.5,2.0
4,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0,7.5,8.0
5,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0,6.0,3.0
6,Angeliño,DF,Manchester City,6.0,423.0,1.0,0.0,0.00,0.00,0.00,0.05,0.26,0.31,83.3,38.3,0.625000,31.0,NaN,NaN
7,Serge Aurier,DF,Tottenham,33.0,2700.0,4.0,1.0,0.03,0.17,0.20,0.02,0.10,0.12,52.5,29.4,0.561224,206.0,5.4,0.0
8,César Azpilicueta,DF,Chelsea,36.0,3229.0,1.0,0.0,0.06,0.17,0.22,0.07,0.16,0.23,50.0,32.7,0.598039,293.0,5.9,1.0
9,Eric Bailly,DF,Manchester Utd,4.0,150.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,23.5,0.000000,13.0,5.0,0.0


In [14]:
defender[defender.isna().any(axis=1)]

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points
1,Max Aarons,DF,Norwich City,36.0,3240.0,7.0,0.0,0.00,0.03,0.03,0.01,0.09,0.11,46.7,27.2,0.562500,221.0,NaN,NaN
6,Angeliño,DF,Manchester City,6.0,423.0,1.0,0.0,0.00,0.00,0.00,0.05,0.26,0.31,83.3,38.3,0.625000,31.0,NaN,NaN
10,Leighton Baines,DF,Everton,8.0,404.0,0.0,0.0,0.00,0.00,0.00,0.01,0.18,0.19,28.6,27.5,0.600000,28.0,NaN,NaN
22,Sam Byram,DF,Norwich City,17.0,1293.0,4.0,0.0,0.00,0.00,0.00,0.08,0.04,0.12,49.0,31.4,0.500000,149.0,NaN,NaN
25,Jonny Castro,DF,Wolves,35.0,2887.0,4.0,0.0,0.06,0.06,0.12,0.04,0.03,0.07,37.8,29.8,0.622449,176.0,NaN,NaN
26,Craig Cathcart,DF,Watford,29.0,2488.0,7.0,0.0,0.00,0.00,0.00,0.01,0.00,0.01,33.3,34.9,0.702703,252.0,NaN,NaN
33,Steve Cook,DF,Bournemouth,29.0,2428.0,6.0,1.0,0.04,0.00,0.04,0.03,0.01,0.04,39.1,33.5,0.590909,314.0,NaN,NaN
36,Charlie Daniels,DF,Bournemouth,2.0,126.0,0.0,0.0,0.00,0.00,0.00,0.00,0.05,0.05,50.0,19.2,0.000000,10.0,NaN,NaN
38,Kevin Danso,DF,Southampton,6.0,277.0,2.0,1.0,0.00,0.32,0.32,0.03,0.07,0.10,62.5,24.4,0.500000,27.0,NaN,NaN
40,Craig Dawson,DF,Watford,29.0,2361.0,6.0,0.0,0.08,0.00,0.08,0.08,0.01,0.09,45.2,39.3,0.633333,255.0,NaN,NaN


In [15]:
defender['ppg'] = defender.apply(lambda row: (row['points']/row['games']),axis=1)
defender['ppc'] = defender.apply(lambda row: (row['points']/row['cost']),axis=1)
defender = defender.dropna().reset_index(drop=True)
defender

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points,ppg,ppc
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0,5.5,0.0,0.000000,0.000000
1,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0,5.5,2.0,0.068966,0.363636
2,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0,5.5,2.0,0.060606,0.363636
3,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0,7.5,8.0,0.210526,1.066667
4,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0,6.0,3.0,0.166667,0.500000
5,Serge Aurier,DF,Tottenham,33.0,2700.0,4.0,1.0,0.03,0.17,0.20,0.02,0.10,0.12,52.5,29.4,0.561224,206.0,5.4,0.0,0.000000,0.000000
6,César Azpilicueta,DF,Chelsea,36.0,3229.0,1.0,0.0,0.06,0.17,0.22,0.07,0.16,0.23,50.0,32.7,0.598039,293.0,5.9,1.0,0.027778,0.169492
7,Eric Bailly,DF,Manchester Utd,4.0,150.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,23.5,0.000000,13.0,5.0,0.0,0.000000,0.000000
8,Fabián Balbuena,DF,West Ham,17.0,1177.0,2.0,0.0,0.08,0.00,0.08,0.14,0.00,0.14,23.1,34.5,0.611111,146.0,4.5,0.0,0.000000,0.000000
9,George Baldock,DF,Sheffield Utd,38.0,3420.0,7.0,0.0,0.05,0.08,0.13,0.02,0.07,0.09,52.1,28.0,0.608108,187.0,5.5,3.0,0.078947,0.545455


In [16]:
defender.to_csv('data/defender_dash.csv',index=False)

In [17]:
defender.shape

(134, 21)